<a href="https://colab.research.google.com/github/djlittle/Facilities-Time-Series-Model-Comparison/blob/master/Facilities_Time_Series_Model_Comparison_(Prediction).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Imports

import pandas as pd
import numpy as np
import operator
import time
import datetime
import re

from scipy import stats

from statsmodels.tsa.statespace.sarimax import SARIMAX

print('Imports completed successfully')

Imports completed successfully


In [0]:
# ----- General options -------
min_wos = 20
min_topic_volume = 5

# ----- User output options -------
verbose = True
print_processing_time = True
print_size = 5
n_print_terms = 20

# ----- Statistical distribution option(s) -------
topic_alpha = 0.1
confidence = 0.95

# ----- SARIMAX model options(s) ----------------
forecast_weeks = 3


In [3]:
# Data load

url = 'https://raw.githubusercontent.com/djlittle/Facilities-Time-Series-Model-Comparison/master/Data/Topic_Time_Series_Data.csv'

count_by_week = pd.read_csv(url)

print(count_by_week.head())

   Week of Year  Topic 1  Topic 2  Topic 3  ...  Topic 7  Topic 8  Topic 9  Topic 10
0             2        0        0        0  ...        0        0        0         0
1             3        0        1        0  ...        0        0        1         1
2             4        0        1        1  ...        0        0        0         2
3             5        0        0        0  ...        1        1        2         2
4             6        0        0        1  ...        1        0        0         1

[5 rows x 11 columns]


In [4]:
count_by_week    # Displaying the dataframe

,Week of Year,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,Topic 10
0,2,0,0,0,0,0,0,0,0,0,0
1,3,0,1,0,0,2,0,0,0,1,1
2,4,0,1,1,1,21,1,0,0,0,2
3,5,0,0,0,0,0,2,1,1,2,2
4,6,0,0,1,0,4,0,1,0,0,1
5,7,0,2,0,0,3,0,0,1,0,2
6,8,1,10,0,43,0,3,1,2,0,1
7,9,3,7,18,21,2,19,9,6,2,4
8,10,5,2,30,19,7,52,9,2,2,8
9,11,1,6,31,6,7,10,13,4,7,7


In [5]:
forecast_df = pd.DataFrame()

topic_forecast_accuracy = []
modeled_topics = []
for topic in count_by_week.columns[1:]:
  # Model fitting
  # ***Options here are to be explored
  if count_by_week[topic].sum() < min_topic_volume: # Removing extremely low volume occurances 
      continue

  modeled_topics.append(topic)
  sarimax = SARIMAX(count_by_week[topic], order=(2, 0, 0), seasonal_order=(1, 1, 1, 4))

  try:
      sarimax_fit = sarimax.fit(disp=False)
  except:
      sarimax_fit = sarimax.fit(disp=False, start_params=[0, 0, 0, 0, 1])

  if verbose:
      # print(yhat)
      print(sarimax_fit.summary())

  # ----------------------- Error estimate ---------------------------------------------------------
  res = sarimax.filter(sarimax_fit.params)
  predict = res.get_prediction()
  predict_mean = predict.predicted_mean
  predict_mean[predict_mean < 0] = 0.00001  # removing erronious negative values
  topic_cbw = count_by_week[topic]
  residuals = predict_mean - topic_cbw
  error = abs(residuals)[6:]/(topic_cbw[6:].mean())
  percent_error = ((error - error.min()) / (error.max() - error.min())).mean()
  topic_forecast_accuracy.append(1-percent_error)

  if forecast_df.empty:
    continue

#   convert forcast accuracy to an array
topic_forecast_accuracy = np.asanyarray(topic_forecast_accuracy)


                                 Statespace Model Results                                
Dep. Variable:                           Topic 1   No. Observations:                   51
Model:             SARIMAX(2, 0, 0)x(1, 1, 1, 4)   Log Likelihood                 -91.231
Date:                           Tue, 28 Jan 2020   AIC                            192.462
Time:                                   21:22:03   BIC                            201.713
Sample:                                        0   HQIC                           195.943
                                            - 51                                         
Covariance Type:                             opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.1013      0.235      0.432      0.666      -0.359       0.561
ar.L2          0.2052      0.438      0.468

In [6]:
print(topic_forecast_accuracy) # Each topic forecast accuracy
print(topic_forecast_accuracy.mean()) # Aggregate forecast accuracy

[0.84043335 0.77308153 0.80652674 0.89202314 0.87632625 0.89046228
 0.69837861 0.76764473 0.78371851 0.74775994]
0.8076355078677031
